# Building the networks

In [ ]:
import pybrain
from pybrain.structure import RecurrentNetwork, FeedForwardNetwork, LinearLayer, SigmoidLayer, FullConnection
from IPython.display import Image
from pybrain.structure import connections
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%pylab inline --no-import-all

Three networks are compared: 
* A FeedForward network "One-step-ahead prediction": **netF**
>The networks has 24 inputs, which means 8 for each step (t, t-1, t-2)
>A Backpropagation trainer is used. 
* A RecurrentNetwork looking 1 step in the past: **netR** 
>This network is a RecurrentNetwork. The trainer is a BackPropTrainer without any configuration, just in the way PyBrain makes it work. It is a Backpropagation through time with one step backward 
* A RecurrentNetwork looking 2 steps in the past: **netR2**
>This is also a RecurrentNetwork trained with Backpropagation through time but changed to two steps backward


In [ ]:
epochs=500

## Creating the layers 

In [ ]:
netF = FeedForwardNetwork()
netR = RecurrentNetwork()
netR2 = RecurrentNetwork()
    
    
inpF = LinearLayer(24)
hidF = SigmoidLayer(24)
outF = LinearLayer(2)

inpR = LinearLayer(8, name='in')
hidR = SigmoidLayer(8, name='hidden')
outR = LinearLayer(2, name='out')

inpR2 = LinearLayer(8, name='in2')
hidR2 = SigmoidLayer(8, name='hidden2')
outR2 = LinearLayer(2, name='out2')
    
netF.addInputModule(inpF)
netF.addModule(hidF)
netF.addOutputModule(outF)


netR.addInputModule(inpR)
netR.addModule(hidR)
netR.addOutputModule(outR)

netR2.addInputModule(inpR2)
netR2.addModule(hidR2)
netR2.addOutputModule(outR2)


## Connecting the layers

In [ ]:
#Feedforward connections

netF.addConnection(FullConnection(inpF,hidF))
netF.addConnection(FullConnection(hidF,outF))
netF.sortModules()

netR.addConnection(FullConnection(netR['in'],netR['hidden'], name='c1'))
netR.addConnection(FullConnection(netR['hidden'],netR['out'], name='c2'))
netR.sortModules()

netR2.addConnection(FullConnection(netR2['in2'],netR2['hidden2'], name='c3'))
netR2.addConnection(FullConnection(netR2['hidden2'],netR2['out2'], name='c4'))
netR2.sortModules()

In [ ]:
#Recurrent connections

netR.addRecurrentConnection(FullConnection(netR['in'],netR['in'], name='cR1'))
netR.addRecurrentConnection(FullConnection(netR['hidden'],netR['hidden'], name='cR2'))
netR.addRecurrentConnection(FullConnection(netR['out'],netR['out'], name='cR3'))
netR.sortModules()

netR2.addRecurrentConnection(FullConnection(netR2['in2'],netR2['in2'], name='cR4'))
netR2.addRecurrentConnection(FullConnection(netR2['hidden2'],netR2['hidden2'], name='cR5'))
netR2.addRecurrentConnection(FullConnection(netR2['out2'],netR2['out2'], name='cR6'))
netR2.sortModules()

# Creating the datasets 

In [ ]:
from pybrain.datasets import SupervisedDataSet

A function to get the data from a CSV file is defined below

In [ ]:
def getcsv( filename ):
    temp = np.fromfile( filename, sep=';' )
    numcols = len( temp )
    del( temp )
    df = pd.read_csv( filename, sep=';', names = [ x+1 for x in range(numcols)] )
    return df

## Importing the files

There are three files needed: 
* The training data
* The validation data
* The test data
    - Collector Greenhouse
    - Reference Greenhouse

In [ ]:
train=getcsv('train.csv')
val = getcsv( 'val.csv' )
test_ref=getcsv('testref.csv')
test_col=getcsv('testcol.csv')

In [ ]:
cols = [ \
    'Record(1)', 'Temperature(1)', 'RelHum(1)', \
    'Ventilation(1)', 'Screening(1)', 'Heating(1)', 'Cooling(1)', \
    'LAI(1)', 'OutTemp(1)', 'OutRelHum(1)', 'OutRad(1)', 'OutWindVel(1)', \
    'HourAngle(1)', 'Declination(1)', 'Elevation(1)', 'RadTheor(1)', \
    'Record(0)', 'Temperature(0)', 'RelHum(0)', \
    'Ventilation(0)', 'Screening(0)', 'Heating(0)', 'Cooling(0)', \
    'LAI(0)', 'OutTemp(0)', 'OutRelHum(0)', 'OutRad(0)', 'OutWindVel(0)', \
    'HourAngle(0)', 'Declination(0)', 'Elevation(0)', 'RadTheor(0)', \
    'Record(-1)', 'Temperature(-1)', 'RelHum(-1)', \
    'Ventilation(-1)', 'Screening(-1)', 'Heating(-1)', 'Cooling(-1)', \
    'LAI(-1)', 'OutTemp(-1)', 'OutRelHum(-1)', 'OutRad(-1)', 'OutWindVel(-1)', \
    'HourAngle(-1)', 'Declination(-1)', 'Elevation(-1)', 'RadTheor(-1)', \
    'Record(-2)', 'Temperature(-2)', 'RelHum(-2)', \
    'Ventilation(-2)', 'Screening(-2)', 'Heating(-2)', 'Cooling(-2)', \
    'LAI(-2)', 'OutTemp(-2)', 'OutRelHum(-2)', 'OutRad(-2)', 'OutWindVel(-2)', \
    'HourAngle(-2)', 'Declination(-2)', 'Elevation(-2)', 'RadTheor(-2)', \
    'Record(-3)', 'Temperature(-3)', 'RelHum(-3)', \
    'Ventilation(-3)', 'Screening(-3)', 'Heating(-3)', 'Cooling(-3)', \
    'LAI(-3)', 'OutTemp(-3)', 'OutRelHum(-3)', 'OutRad(-3)', 'OutWindVel(-3)', \
    'HourAngle(-3)', 'Declination(-3)', 'Elevation(-3)', 'RadTheor(-3)', \
    ]
train.columns = cols
val.columns = cols
test_ref.columns=cols
test_col.columns=cols

This data looks like:

In [ ]:
train.head()

## Data normalization

In order to enhance the comparison and the error calculation process, the data are normalized. This makes posible to have a common format doesn't matter the units. 
The functions to normalize and de-normalize the data are shown below: 

### max & min

In [ ]:
indexm=(["maximos","minimos"])
mm = pd.DataFrame(columns=train.columns,index=indexm)

for n in range(1,train.shape[1]):
    max1=max(train[train.columns[n]])
    max2=max(val[val.columns[n]])
    mm[mm.columns[n]]["maximos"]=max(max1,max2)
    
    min1=min(train[train.columns[n]])
    min2=min(val[val.columns[n]])
    mm[mm.columns[n]]["minimos"]=min(min1,min2)


In [ ]:
def norm(x,maxi,mini):
    Vnorm=(2*((x-mini)/(maxi-mini)))-1
    return Vnorm  


In [ ]:
def a_norm(y,maxi,mini):
    Anorm=(((y+1)*(maxi-mini))/2)+mini
    return Anorm

In [ ]:
Ntrain = pd.DataFrame(columns=train.columns)
Nval = pd.DataFrame(columns=val.columns)
Ntest_col = pd.DataFrame(columns=test_col.columns)
Ntest_ref = pd.DataFrame(columns=test_ref.columns)

In [ ]:
for n in range(1,train.shape[1]):
    ar=train[train.columns[n]]
    Ntrain[Ntrain.columns[n]]=norm(ar,mm[mm.columns[n]]["maximos"],mm[mm.columns[n]]["minimos"])
    
for n in range(1,val.shape[1]):
    ar=val[val.columns[n]]
    Nval[Nval.columns[n]]=norm(ar,mm[mm.columns[n]]["maximos"],mm[mm.columns[n]]["minimos"])
    
for n in range(1,test_col.shape[1]):
    ar=test_col[test_col.columns[n]]
    Ntest_col[Ntest_col.columns[n]]=norm(ar,mm[mm.columns[n]]["maximos"],mm[mm.columns[n]]["minimos"])

for n in range(1,test_ref.shape[1]):
    ar=test_ref[test_ref.columns[n]]
    Ntest_ref[Ntest_ref.columns[n]]=norm(ar,mm[mm.columns[n]]["maximos"],mm[mm.columns[n]]["minimos"])
    

##The first column with the record number remains the same     
Ntrain[Ntrain.columns[0]]=train[train.columns[0]]
Nval[Nval.columns[0]]=val[val.columns[0]]
Ntest_col[Ntest_col.columns[0]]=test_col[test_col.columns[0]]
Ntest_ref[Ntest_ref.columns[0]]=test_ref[test_ref.columns[0]]

So now, the data is going from -1 to 1, doesn't matter the units:

In [ ]:
Ntest_col.head()

## Training, validation and test datasets

Now, the datasets for training and validation are created with the standarized data from the files.
>There are training and validation datasets for the FeedForward network and training and validation datasets for the Recurrent Networks
>This is due to the different number of inputs for each different typology of network

### Feedforward Networks

In [ ]:
#________________________________________________Training_______________________________________________________

s=np.array([Ntrain['Temperature(-2)'],Ntrain['RelHum(-2)'],Ntrain['Ventilation(-2)'],Ntrain['Screening(-2)'], \
           Ntrain['HourAngle(-2)'],Ntrain['Declination(-2)'],Ntrain['Elevation(-2)'],Ntrain['RadTheor(-2)'], \
           Ntrain['Temperature(-1)'],Ntrain['RelHum(-1)'],Ntrain['Ventilation(-1)'],Ntrain['Screening(-1)'], \
           Ntrain['HourAngle(-1)'],Ntrain['Declination(-1)'],Ntrain['Elevation(-1)'],Ntrain['RadTheor(-1)'], \
           Ntrain['Temperature(0)'],Ntrain['RelHum(0)'],Ntrain['Ventilation(0)'],Ntrain['Screening(0)'], \
           Ntrain['HourAngle(0)'],Ntrain['Declination(0)'],Ntrain['Elevation(0)'],Ntrain['RadTheor(0)']]).T

t=np.array([Ntrain['Temperature(1)'],Ntrain['RelHum(1)']]).T


#______________________________________________Validation_______________________________________________________

sval=np.array([Nval['Temperature(-2)'],Nval['RelHum(-2)'],Nval['Ventilation(-2)'],Nval['Screening(-2)'], \
           Nval['HourAngle(-2)'],Nval['Declination(-2)'],Nval['Elevation(-2)'],Nval['RadTheor(-2)'], \
           Nval['Temperature(-1)'],Nval['RelHum(-1)'],Nval['Ventilation(-1)'],Nval['Screening(-1)'], \
           Nval['HourAngle(-1)'],Nval['Declination(-1)'],Nval['Elevation(-1)'],Nval['RadTheor(-1)'], \
           Nval['Temperature(0)'],Nval['RelHum(0)'],Nval['Ventilation(0)'],Nval['Screening(0)'], \
           Nval['HourAngle(0)'],Nval['Declination(0)'],Nval['Elevation(0)'],Nval['RadTheor(0)']]).T

tval=np.array([Nval['Temperature(1)'],Nval['RelHum(1)']]).T     


#________________________________________________Test____________________________________________________________

stest_col=np.array([Ntest_col['Temperature(-2)'],Ntest_col['RelHum(-2)'],Ntest_col['Ventilation(-2)'],Ntest_col['Screening(-2)'], \
           Ntest_col['HourAngle(-2)'],Ntest_col['Declination(-2)'],Ntest_col['Elevation(-2)'],Ntest_col['RadTheor(-2)'], \
           Ntest_col['Temperature(-1)'],Ntest_col['RelHum(-1)'],Ntest_col['Ventilation(-1)'],Ntest_col['Screening(-1)'], \
           Ntest_col['HourAngle(-1)'],Ntest_col['Declination(-1)'],Ntest_col['Elevation(-1)'],Ntest_col['RadTheor(-1)'], \
           Ntest_col['Temperature(0)'],Ntest_col['RelHum(0)'],Ntest_col['Ventilation(0)'],Ntest_col['Screening(0)'], \
           Ntest_col['HourAngle(0)'],Ntest_col['Declination(0)'],Ntest_col['Elevation(0)'],Ntest_col['RadTheor(0)']]).T

ttest_col=np.array([Ntest_col['Temperature(1)'],Ntest_col['RelHum(1)']]).T     


stest_ref=np.array([Ntest_ref['Temperature(-2)'],Ntest_ref['RelHum(-2)'],Ntest_ref['Ventilation(-2)'],Ntest_ref['Screening(-2)'], \
           Ntest_ref['HourAngle(-2)'],Ntest_ref['Declination(-2)'],Ntest_ref['Elevation(-2)'],Ntest_ref['RadTheor(-2)'], \
           Ntest_ref['Temperature(-1)'],Ntest_ref['RelHum(-1)'],Ntest_ref['Ventilation(-1)'],Ntest_ref['Screening(-1)'], \
           Ntest_ref['HourAngle(-1)'],Ntest_ref['Declination(-1)'],Ntest_ref['Elevation(-1)'],Ntest_ref['RadTheor(-1)'], \
           Ntest_ref['Temperature(0)'],Ntest_ref['RelHum(0)'],Ntest_ref['Ventilation(0)'],Ntest_ref['Screening(0)'], \
           Ntest_ref['HourAngle(0)'],Ntest_ref['Declination(0)'],Ntest_ref['Elevation(0)'],Ntest_ref['RadTheor(0)']]).T

ttest_ref=np.array([Ntest_ref['Temperature(1)'],Ntest_ref['RelHum(1)']]).T     


### Recurrent Networks

In [ ]:
#__________________________________________Training______________________________________________________________

sR=np.array([Ntrain['Temperature(0)'],Ntrain['RelHum(0)'],Ntrain['Ventilation(0)'],Ntrain['Screening(0)'], \
           Ntrain['HourAngle(0)'],Ntrain['Declination(0)'],Ntrain['Elevation(0)'],Ntrain['RadTheor(0)']]).T

tR=np.array([Ntrain['Temperature(1)'],Ntrain['RelHum(1)']]).T


#__________________________________________Validation_____________________________________________________________

svalR=np.array([Nval['Temperature(0)'],Nval['RelHum(0)'],Nval['Ventilation(0)'],Nval['Screening(0)'], \
           Nval['HourAngle(0)'],Nval['Declination(0)'],Nval['Elevation(0)'],Nval['RadTheor(0)']]).T

tvalR=np.array([Nval['Temperature(1)'],Nval['RelHum(1)']]).T     


#____________________________________________Test__________________________________________________________________

stest_colR=np.array([Ntest_col['Temperature(0)'],Ntest_col['RelHum(0)'],Ntest_col['Ventilation(0)'],Ntest_col['Screening(0)'], \
           Ntest_col['HourAngle(0)'],Ntest_col['Declination(0)'],Ntest_col['Elevation(0)'],Ntest_col['RadTheor(0)']]).T

ttest_colR=np.array([Ntest_col['Temperature(1)'],Ntest_col['RelHum(1)']]).T  


stest_refR=np.array([Ntest_ref['Temperature(0)'],Ntest_ref['RelHum(0)'],Ntest_ref['Ventilation(0)'],Ntest_ref['Screening(0)'], \
           Ntest_ref['HourAngle(0)'],Ntest_ref['Declination(0)'],Ntest_ref['Elevation(0)'],Ntest_ref['RadTheor(0)']]).T

ttest_refR=np.array([Ntest_ref['Temperature(1)'],Ntest_ref['RelHum(1)']]).T  

In [ ]:
##Length of the data

data_length=len(s)
length_Val=len(sval)
length_Test=len(stest_col)
print('Ttraining:',data_length,'\nValidation:',length_Val,'\nTest:',length_Test)

In [ ]:
#Dataset for training

dataF = SupervisedDataSet(24,2)
dataR = SupervisedDataSet(8,2)

for m in range(0,data_length):
    dataF.addSample(s[m],t[m])  
    dataR.addSample(sR[m],tR[m])

# Training the networks

In [ ]:
from pybrain.supervised import BackpropTrainer

Two functions are defined below for the RMSE and MSE calculations

In [ ]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

def mse(predictions, targets):
    return ((predictions - targets) ** 2).mean()

## FeedForward network

The network is trained with BackpropTrainer for 1000 epochs. Inside the training loop, three errors are calculated:
* The one given by the trainer **errors1**
* The MSE calculated with the same training data **errors2**
* The validation MSE **errorsVal**

In [ ]:
trainer = BackpropTrainer(netF, dataF, learningrate = 0.001, momentum = 0.99, verbose=False)

errors1 = np.array([])
errors2=np.array([])
errorsVal=np.array([])

#_______________________________________Trainer error_________________________________________________

for epoch in range(0,epochs):
    temp = trainer.train()
    errors1=np.append(errors1,temp)
    
#______________________________________calculated MSE________________________________________________

    er2=np.array([])
    for inps in s:
        j=netF.activate(inps)
        er2=np.append(er2,j)
    er2=np.reshape(er2,[data_length,2])
    errors2=np.append(errors2,mse(er2,t))
    

#_______________________________________validation MSE_______________________________________________
    
    er_Val=np.array([])
    for inps in sval:
        k=netF.activate(inps)
        er_Val=np.append(er_Val,k)
    er_Val=np.reshape(er_Val,[length_Val,2])
    errorsVal=np.append(errorsVal,mse(er_Val,tval))
    
    
   

The given and the calculated training errors are shown below

In [ ]:
print('   errors1','     errors2', 'Difference e1/e2')
for i in range(0,epochs):
    print(i, errors1[i],'->',errors2[i],'    ', errors1[i]/errors2[i])

The next figure demonstrates the difference between both MSE

In [ ]:
fig1, ax1 = plt.subplots(1, sharex=True, figsize=(35,8))
ax1.plot(errors1,color='blue', linewidth=1.0, label='Errors1')
#ax1.plot(errors2,color='red', marker='.', linestyle=' ',linewidth=1.0, label='Errors2')
ax1.plot(errors2,color='red',linewidth=1.0, label='Errors2')
ax1.set_title('Given and Calculated Errors for Feedforward Network');
#ax.set_xlim([0,20])
#ax.set_ylim([0,30])
ax1.legend(loc='upper right')
fig1.savefig("figure1.pdf") # save as pdf

## Recurrent Network offset=1

Like the FeedForward network, this network is trained with BackpropTrainer through time for 1000 epochs and one step backwards. Inside the training loop, three errors are calculated:
* The one given by the trainer **errors1_R**
* The MSE calculated with the same training data **errors2_R**
* The validation MSE **errorsVal_R**

In [ ]:
trainerR = BackpropTrainer(netR, dataR, learningrate = 0.001, momentum = 0.99, verbose = False)

errors1_R = np.array([])
errors2_R=np.array([])
errorsVal_R=np.array([])

#_______________________________________Trainer error_________________________________________________

for epoch in range(0,epochs):
    temp = trainerR.train()
    errors1_R=np.append(errors1_R,temp)
    
#______________________________________calculated MSE________________________________________________

    er2_R=np.array([])
    for inps_R in sR:
        netR.reset()
        j_R=netR.activate(inps_R)
        er2_R=np.append(er2_R,j_R)
    er2_R=np.reshape(er2_R,[data_length,2])
    errors2_R=np.append(errors2_R,mse(er2_R,tR))
    
#_______________________________________validation MSE_______________________________________________
    
    er_Val_R=np.array([])
    for inps in svalR:
        netR.reset()
        k_R=netR.activate(inps)
        er_Val_R=np.append(er_Val_R,k_R)
    er_Val_R=np.reshape(er_Val_R,[length_Val,2])
    errorsVal_R=np.append(errorsVal_R,mse(er_Val_R,tvalR))
    

The given and the calculated training errors are shown below

In [ ]:
print('   errors1_R','     errors2_R', 'Difference e1/e2')
for i in range(0,epochs):
    print(i,errors1_R[i],'->',errors2_R[i],'    ', errors1_R[i]/errors2_R[i])

The next figure demonstrates the difference between both MSE

In [ ]:
fig2, ax2 = plt.subplots(1, sharex=True, figsize=(35,8))
ax2.plot(errors1_R,color='blue', linewidth=1.0, label='Errors1_R')
ax2.plot(errors2_R,color='red', linewidth=1.0, label='Errors2_R')
ax2.set_title('Given and Calculated Errors for the Recurrent Network 1');
#ax.set_xlim([0,20])
#ax2.set_ylim([0,0.05])
ax2.legend(loc='upper right')
fig2.savefig("figure2.pdf") # save as pdf

## Recurrent Network offset=2

The default value for the offset is 1. In order to make the network look two steps in the past is necesary to change this value

In [ ]:
netR2.maxoffset=2

As in the two networks shown above, this network is trained with BackpropTrainer through time for 1000 epochs but with two step backwards. Inside the training loop, three errors are calculated:
* The one given by the trainer **errors1_R2**
* The MSE calculated with the same training data **errors2_R2**
* The validation MSE **errorsVal_R2**

In [ ]:
trainerR2 = BackpropTrainer(netR2, dataR, learningrate = 0.001, momentum = 0.99, verbose = False)


errors1_R2 = np.array([])
errors2_R2=np.array([])
errorsVal_R2=np.array([])

#_______________________________________Trainer error_________________________________________________

for epoch in range(0,epochs):
    temp = trainerR2.train()
    errors1_R2=np.append(errors1_R2,temp)
    
#______________________________________calculated MSE________________________________________________

    er2_R2=np.array([])
    for inpsR2 in sR:
        netR2.reset()
        j_R2=netR2.activate(inpsR2)
        er2_R2=np.append(er2_R2,j_R2)
    er2_R2=np.reshape(er2_R2,[data_length,2])
    errors2_R2=np.append(errors2_R2,mse(er2_R2,tR))
    
#_______________________________________validation MSE_______________________________________________
    
    er_Val_R2=np.array([])
    for inps in svalR:
        netR2.reset()
        k_R2=netR2.activate(inps)
        er_Val_R2=np.append(er_Val_R2,k_R2)
    er_Val_R2=np.reshape(er_Val_R2,[length_Val,2])
    errorsVal_R2=np.append(errorsVal_R2,mse(er_Val_R2,tvalR))
    
    

The given and the calculated training errors are shown below

In [ ]:
print('   errors1_R2','     errors2_R2', 'Difference e1/e2')
for i in range(0,epochs):
    print(i,errors1_R2[i],'->',errors2_R2[i],'    ', errors1_R2[i]/errors2_R2[i])

The next figure demonstrates the difference between both MSE

In [ ]:
fig3, ax3 = plt.subplots(1, sharex=True, figsize=(35,8))
ax3.plot(errors1_R2,color='blue', linewidth=1.0, label='Errors1_R2')
ax3.plot(errors2_R2,color='red', linewidth=1.0, label='Errors2_R2')
ax3.set_title('Given and Calculated Errors for the Recurrent Network 2');
#ax3.set_xlim([20,40])
#ax3.set_ylim([0,.030])
ax3.legend(loc='upper right')
fig3.savefig("figure3.pdf") # save as pdf

# Comparision between the three tipologies of Neural Networks

In the figure below it is shown the MSE calculated for each Network

In [ ]:
fig4, ax4 = plt.subplots(1, sharex=True, figsize=(35,8))
ax4.plot(errors2,color='blue', linewidth=1.0, label='FeedForward')
ax4.plot(errors2_R,color='magenta', linewidth=1.0, label='RecurrentNetwork')
ax4.plot(errors2_R2,color='green', linewidth=1.0,label='RecurrentNetwork 2')
ax4.set_title('MSE ');
#ax4.set_xlim([900,1000])
#ax4.set_ylim([0,0.002])
ax4.legend(loc='upper right')
fig4.savefig("figure4.pdf") # save as pdf

## Validation graphics

**Feedforward Network**

In [ ]:
fig5, ax5 = plt.subplots(1, sharex=True, figsize=(35,8))
ax5.plot(errors2,color='red', linewidth=1.0, label='Training Error')
ax5.plot(errorsVal,color='magenta', linewidth=1.0,label='Validation Error')
ax5.set_title('Training vs Validation Error: FeedForward');
#ax5.set_xlim([90,100])
#ax5.set_ylim([0,.1])
ax5.legend(loc='upper right')
fig5.savefig("figure5.pdf") # save as pdf

**Recurrent Network offset=1**

In [ ]:
fig6, ax6 = plt.subplots(1, sharex=True, figsize=(35,8))
ax6.plot(errors2_R,color='red', linewidth=1.0,label='Training Error')
ax6.plot(errorsVal_R,color='magenta', linewidth=1.0,label='Validation Error')
ax6.set_title('Training vs Validation Error: RecurrentNetwork 1');
#ax6.set_xlim([0,200])
#ax6.set_ylim([0,100])
ax6.legend(loc='upper right')
fig6.savefig("figure6.pdf") # save as pdf

**Recurrent Network offset=2**

In [ ]:
fig7, ax7 = plt.subplots(1, sharex=True, figsize=(35,8))
ax7.plot(errors2_R2,color='red', linewidth=1.0,label='Training Error')
ax7.plot(errorsVal_R2,color='magenta', linewidth=1.0,label='Validation Error')
ax7.set_title('Training vs Validation Error: RecurrentNetwork 2');
#ax7.set_xlim([0,200])
#ax7.set_ylim([0,100])
ax7.legend(loc='upper right')
fig7.savefig("figure7.pdf") # save as pdf

# Testing the networks

## Feedforward Network

In [ ]:
#_______________________________Collector Greenhouse__________________________

j=np.array([])
for inps in (stest_col):
    netF.reset()
    j=np.append(j,netF.activate( inps ))

j=np.reshape(j,[length_Test,2])    

errorTest_col=mse(j,ttest_col)

temperature_errorTest_col=mse(j[0],ttest_col[0])
humidity_errorTest_col=mse(j[1],ttest_col[1])


#______________________________Reference Greenhouse___________________________

j=np.array([])
for inps in (stest_ref):
    netF.reset()
    j=np.append(j,netF.activate( inps ))
    
j=np.reshape(j,[length_Test,2])    

errorTest_ref=mse(j,ttest_ref)

temperature_errorTest_ref=mse(j[0],ttest_ref[0])
humidity_errorTest_ref=mse(j[1],ttest_ref[1])


## Recurrent Network, offset=1

In [ ]:
#_______________________________Collector Greenhouse__________________________

j=np.array([])
for inps in (stest_colR):
    netR.reset()
    j=np.append(j,netR.activate( inps ))

j=np.reshape(j,[length_Test,2])    

errorTestR_col=mse(j,ttest_colR)

temperature_errorTestR_col=mse(j[0],ttest_colR[0])
humidity_errorTestR_col=mse(j[1],ttest_colR[1])

#______________________________Reference Greenhouse___________________________

j=np.array([])
for inps in (stest_refR):
    netR.reset()
    j=np.append(j,netR.activate( inps ))
    
j=np.reshape(j,[length_Test,2])    

errorTestR_ref=mse(j,ttest_refR)

temperature_errorTestR_ref=mse(j[0],ttest_refR[0])
humidity_errorTestR_ref=mse(j[1],ttest_refR[1])



## Recurrent Network, offset=2

In [ ]:
#_______________________________Collector Greenhouse__________________________

j=np.array([])
for inps in (stest_colR):
    netR2.reset()
    j=np.append(j,netR2.activate( inps ))

j=np.reshape(j,[length_Test,2])    

errorTestR2_col=mse(j,ttest_colR)

temperature_errorTestR2_col=mse(j[0],ttest_colR[0])
humidity_errorTestR2_col=mse(j[1],ttest_colR[1])

#______________________________Reference Greenhouse___________________________

j=np.array([])
for inps in (stest_refR):
    netR2.reset()
    j=np.append(j,netR2.activate( inps ))  
j=np.reshape(j,[length_Test,2])    

errorTestR2_ref=mse(j,ttest_refR)

temperature_errorTestR2_ref=mse(j[0],ttest_refR[0])
humidity_errorTestR2_ref=mse(j[1],ttest_refR[1])


## Comparition

### Comparing the three Networks

In [ ]:
fig8, ax8 = plt.subplots(1, sharex=True, figsize=(10,7))
a=3

collector=[errorTest_col,errorTestR_col,errorTestR2_col]
reference=[errorTest_ref,errorTestR_ref,errorTestR2_ref]

ind = np.arange(a)  # the x locations for the groups
width = 0.15 # the width of the bars

rects1 = ax8.bar(0.5*ind, collector, width, color='green')
rects2 = ax8.bar(0.5*ind+width, reference, width, color='blue')

# add some text for labels, title and axes ticks
ax8.set_title('MSE for the three typologies of Neural Networks')
ax8.set_ylabel('MSE')
ax8.set_xticks(0.5*ind +width)
ax8.set_xticklabels(('FeedForward',  'Recurrent', 'Recurrent 2'))
ax8.set_ylim([0,0.25])
ax8.legend(('Collector', 'Reference'),loc='upper right',)
fig8.savefig("figure8.pdf") # save as pdf

plt.show()

### Feedforward Network

In [ ]:
fig9, ax9 = plt.subplots(1, sharex=True, figsize=(7,5))

a=2
collector=[temperature_errorTest_col,humidity_errorTest_col]
reference=[temperature_errorTest_ref,humidity_errorTest_ref]

ind = np.arange(a)  # the x locations for the groups
width = 0.15 # the width of the bars


rects1 = ax9.bar(0.5*ind, collector, width, color='orange')
rects2 = ax9.bar(0.5*ind+width, reference, width, color='yellow')

# add some text for labels, title and axes ticks
ax9.set_title('MSE for the Feedforward Neural Network')
ax9.set_ylabel('MSE')
ax9.set_xticks(0.5*ind +width)
ax9.set_xticklabels(('Temperature',  'Humidity'))
ax9.set_ylim([0,0.01])
ax9.legend(('Collector', 'Reference'),loc='upper right',)
fig9.savefig("figure9.pdf") # save as pdf

plt.show()

### Recurrent Network, offset=1

In [ ]:
fig10, ax10 = plt.subplots(1, sharex=True, figsize=(7,5))

a=2
collector=[temperature_errorTestR_col,humidity_errorTestR_col]
reference=[temperature_errorTestR_ref,humidity_errorTestR_ref]

ind = np.arange(a)  # the x locations for the groups
width = 0.15 # the width of the bars


rects1 = ax10.bar(0.5*ind, collector, width, color='orange')
rects2 = ax10.bar(0.5*ind+width, reference, width, color='yellow')

# add some text for labels, title and axes ticks
ax10.set_title('MSE for the Recurrent Neural Network, offset=1')
ax10.set_ylabel('MSE')
ax10.set_xticks(0.5*ind +width)
ax10.set_xticklabels(('Temperature',  'Humidity'))
ax10.set_ylim([0,0.1])
ax10.legend(('Collector', 'Reference'),loc='upper right',)
fig10.savefig("figure10.pdf") # save as pdf

plt.show()

### Recurrent Network, offset=2

In [ ]:
fig11, ax11 = plt.subplots(1, sharex=True, figsize=(7,5))

a=2
collector=[temperature_errorTestR2_col,humidity_errorTestR2_col]
reference=[temperature_errorTestR2_ref,humidity_errorTestR2_ref]

ind = np.arange(a)  # the x locations for the groups
width = 0.15 # the width of the bars


rects1 = ax11.bar(0.5*ind, collector, width, color='orange')
rects2 = ax11.bar(0.5*ind+width, reference, width, color='yellow')

# add some text for labels, title and axes ticks
ax11.set_title('MSE for the Recurrent Neural Network, offset=2')
ax11.set_ylabel('MSE')
ax11.set_xticks(0.5*ind +width)
ax11.set_xticklabels(('Temperature',  'Humidity'))
ax11.set_ylim([0,0.1])
ax11.legend(('Collector', 'Reference'),loc='upper right',)
fig11.savefig("figure11.pdf") # save as pdf

plt.show()